In [1]:
#import required libraries
!pip install beautifulsoup4
import pandas as pd
from bs4 import BeautifulSoup
import requests
import urllib.request
import time

In [2]:
# List to store scraped data - look at website orientation

orator=[]
stated_date=[]
text=[]
author=[]
published_date=[]
label=[]


In [3]:
#Assigning request and the data need to scraped from Politifact website

def data_website (pages):
    page=str(pages)
    website_URL = 'https://www.politifact.com/factchecks/list/?page=' + page        # URL
    data= requests.get(website_URL)                                                 # Requesting access
    html = BeautifulSoup(data.text, 'html.parser')                                  # Parsing the website
    text_orator = html.find_all('div',{'class':'m-statement__meta'})                # Locating orator name
    text_stated_date = html.find_all('div',{'class':'m-statement__desc'})           # Locating stated date
    text_text = html.find_all('div',{'class':'m-statement__quote'})                 # Locating the statement 
    text_author = html.find_all('footer',{'class':'m-statement__footer'})           # Locating the author
    text_label = html.find_all('div',{'class':'m-statement__meter'})                # Locating the label
    
# Looping for all Location    

# a) Loop for text_orator

    for i in text_orator:
        text1 = i.find_all('a')
        orator_text = text1[0].text.strip()
        orator.append(orator_text)
    
# b) Loop for text_stated_date

    for i in text_stated_date:
        text2 = i.text.strip()
        text_split = text2.split()
        day = text_split[3]
        month = text_split[2]
        year = text_split[4]
        date = day + ' ' + month + ' ' + year
        stated_date.append(date)
    
# c) Loop for text_text

    for i in text_text:
        text3 = i.find_all('a')
        texts = text3[0].text.strip()
        text.append(texts)
        
# d) Loop for text_author & published_date

    for i in text_author:
        text4 = i.text.strip()
        author_text = text4.split()
        day1 = author_text[5]
        month1 = author_text[4]
        year1 = author_text[6]
        date1 = day1 + ' ' + month1 + ' ' + year1
        first_name = author_text[1]
        second_name = author_text[2] 
        full_name = first_name + ' ' + second_name
        author.append(full_name)
        published_date.append(date1)
        
# e) Loop for text_label

    for i in text_label: 
        text5 = i.find('div',{'class':'c-image'}).find('img').get('alt')
        label.append(text5)
        


In [4]:
# Looping for number of pages

n=160
for i in range(1,n):
    data_website(i)

In [5]:
# Creating dataframe for extracted data

data_x=pd.DataFrame(columns = ['orator','stated_date','text','author','published_date','label'])
data_x['orator']=orator
data_x['stated_date']=stated_date
data_x['text']=text
data_x['author']=author
data_x['published_date']=published_date
data_x['label']=label
data_x.head()

,orator,stated_date,text,author,published_date,label
0,TikTok posts,"17, August 2023","El incendio forestal en Maui, Hawai, fue provo...",Maria Briceño,"25, August 2023",false
1,Facebook posts,"22, August 2023",Former President Barack Obama “reacts to chef’...,Ciara O'Rourke,"24, August 2023",false
2,Facebook posts,"23, August 2023",“Military arrests FEMA Deputy Administrator Er...,Ciara O'Rourke,"24, August 2023",pants-fire
3,Vivek Ramaswamy,"23, August 2023",“The reality is more people are dying of bad c...,Jeff Cercone,"24, August 2023",pants-fire
4,Instagram posts,"20, August 2023",U.S. Navy SEALs rescued 807 children who were ...,Ciara O'Rourke,"24, August 2023",false


In [6]:
data_x['label'].unique()

array(['false', 'pants-fire', 'barely-true', 'half-true', 'true',
       'mostly-true', 'half-flip', 'full-flop', 'no-flip'], dtype=object)

In [7]:
def labels(row):
    
    if(row['label'] =='true'):
        return 'True'
    
    elif (row['label'] =='mostly-true'):
        return 'True'
    
    elif (row['label'] =='half-true'):
        return 'True'
    
    else:
        return 'False'

    
data_x['label']=data_x.apply(labels,axis=1)
data_x.head()
       

,orator,stated_date,text,author,published_date,label
0,TikTok posts,"17, August 2023","El incendio forestal en Maui, Hawai, fue provo...",Maria Briceño,"25, August 2023",False
1,Facebook posts,"22, August 2023",Former President Barack Obama “reacts to chef’...,Ciara O'Rourke,"24, August 2023",False
2,Facebook posts,"23, August 2023",“Military arrests FEMA Deputy Administrator Er...,Ciara O'Rourke,"24, August 2023",False
3,Vivek Ramaswamy,"23, August 2023",“The reality is more people are dying of bad c...,Jeff Cercone,"24, August 2023",False
4,Instagram posts,"20, August 2023",U.S. Navy SEALs rescued 807 children who were ...,Ciara O'Rourke,"24, August 2023",False


In [8]:
data_x.shape

(4770, 6)

In [9]:
data_x.to_excel('LabelData2_Politifact.xlsx', index=False)